In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 2.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=85f5969ecef410136ac39f3ba783cf1800fab3216c42f54b0532c4b6d5f0010a
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=ef9ca7c0a18e06e45adcfcf0ce5efc43411e135b797eafe1f779732cfca856b0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_image,test_label)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/225.0
test_image=test_image/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_image = test_image.reshape(len(test_image),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16),
                                kernel_size=hp.Choice('conv_1_kernel', values =[3,5]),
                                activation= 'relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter',min_value=32, max_value=64, step=16),
                                kernel_size=hp.Choice('conv_2_kernel', values =[3,5]),
                                activation= 'relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                                activation= 'relu'
                            ),
                            keras.layers.Dense(10, activation='softmax')#output layer

  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashon")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 21m 22s]
val_accuracy: 0.09849999845027924

Best val_accuracy So Far: 0.8083333373069763
Total elapsed time: 01h 25m 00s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |96                
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |32                
conv_2_kernel     |3                 |5                 
dense_1_units     |128               |80                
learning_rate     |0.001             |0.001             

Epoch 1/3
1484/1688 [=========================>....] - ETA: 31s - loss: 0.6969 - accuracy: 0.7436

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()